In [ ]:
#Dataset
# https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset      
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report


In [ ]:

# Set the seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data directories
train_dir = "dataset/train"
valid_dir = "dataset/valid"



In [ ]:
from PIL import Image

image_path = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\train\Apple___Apple_scab\0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.JPG'  # Replace with the actual path to your image

# Open the image
image = Image.open(image_path)

# Get the size of the image
img_width, img_height = image.size

print("Image size: {} x {}".format(img_width, img_height))

num_classes = 38


In [ ]:

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_dir = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\train'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

valid_dir = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\valid'

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
from tensorflow.keras.applications import DenseNet121
# Load the pre-trained DenseNet201 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size
)


In [ ]:
model.save('trained.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained model
model = tf.keras.models.load_model('trained.h5')

# Evaluation

loss, accuracy = model.evaluate(valid_generator)
print('Accuracy: {:.2f}%'.format(accuracy * 100))
